In [1]:
import nest_asyncio
# Allows nested asyncio event loops - Jupyter already runs in an event loop,
# so nest_asyncio lets us use asyncio.run() within this environment.
nest_asyncio.apply()

In [2]:
from haystack import Pipeline
from haystack.components.agents import Agent
from haystack_integrations.components.generators.ollama import OllamaChatGenerator
from haystack.components.tools import ToolInvoker
from haystack.components.converters import OutputAdapter
from haystack.dataclasses import ChatMessage

In [3]:
# llm = OllamaChatGenerator(model="qwen2.5:32b", timeout=90, url="http://localhost:11434", streaming_callback=lambda chunk: print(chunk.content, end="", flush=True))

In [ ]:
# messages = [
#     ChatMessage.from_user("What's Natural Language Processing?"),
#     ChatMessage.from_system(
#         "Natural Language Processing (NLP) is a field of computer science and artificial "
#         "intelligence concerned with the interaction between computers and human language"
#     ),
#     ChatMessage.from_user("How do I get started?"),
# ]

# response = llm.run(messages, generation_kwargs={"temperature": 0.2})

# print(response["replies"][0].text)

Natural Language Processing (NLP) is a branch of Artificial Intelligence that focuses on enabling computers to understand, interpret, and generate human languages in a valuable way. It involves the development of algorithms and models that can process large amounts of natural language data to perform tasks such as text classification, sentiment analysis, machine translation, speech recognition, and more.

To get started with NLP, you can follow these steps:

1. **Learn the Basics**: Start by understanding the fundamental concepts in linguistics and computer science relevant to NLP. This

In [3]:
from haystack_integrations.tools.mcp import MCPToolset, StdioServerInfo, SSEServerInfo

In [4]:
server_info = StdioServerInfo(command="uv", args=["run", "--with", "biomcp-python", "biomcp", "run"])

In [8]:
# Create the toolset - this will automatically discover all available tools
mcp_toolset = MCPToolset(server_info)
pipeline = Pipeline()
pipeline.add_component("llm",  OllamaChatGenerator(model="mistral-small3.1", timeout=300, url="http://localhost:11434", tools=mcp_toolset, streaming_callback=lambda chunk: print(chunk.content, end="", flush=True)))
pipeline.add_component("tool_invoker", ToolInvoker(tools=mcp_toolset))
pipeline.add_component(
    "adapter",
    OutputAdapter(
        template="{{ initial_msg + initial_tool_messages + tool_messages }}",
        output_type=list[ChatMessage],
        unsafe=True,
    ),
)
pipeline.add_component("response_llm",  OllamaChatGenerator(model="mistral-small3.1", timeout=300, url="http://localhost:11434", streaming_callback=lambda chunk: print(chunk.content, end="", flush=True)))
pipeline.connect("llm.replies", "tool_invoker.messages")
pipeline.connect("llm.replies", "adapter.initial_tool_messages")
pipeline.connect("tool_invoker.tool_messages", "adapter.tool_messages")
pipeline.connect("adapter.output", "response_llm.messages")

user_input = "Create a short report on clinical trails with Diclofenac."  # can be any city
user_input_msg = ChatMessage.from_user(text=user_input)

result = pipeline.run({"llm": {"messages": [user_input_msg]}, "adapter": {"initial_msg": [user_input_msg]}})

print(result["response_llm"])

Unsafe mode is enabled. This allows execution of arbitrary code in the Jinja template. Use this only if you trust the source of the template.


PipelineRuntimeError: The following component failed to run:
Component name: 'llm'
Component type: 'OllamaChatGenerator'
Error: Ollama does not support tools and streaming at the same time. Please choose one.

In [ ]:
result